In [2]:
import pandas as pd
import re
import openai

In [19]:
# Replace 'your_file.csv' with the path to your CSV file
df = pd.read_csv('data.csv')

In [37]:
def extract_title_enhanced(case_text):
    case_text = str(case_text)
    
    # First attempt with the original pattern
    match = re.search(r'\[\s*(G\.\s*R\.\s*Nos?\.).*?\]\s*.*?(J\.\:|PER\s*CURIAM\:)', case_text, re.DOTALL)
    if match:
        return match.group(0).strip()
    
    # Second attempt with the refined pattern
    match = re.search(
        r'\[\s*((G\.\s*R\.\s*No\.?\s*L?-?\d+\.?)|(\d+[-\d]*))\.\s*.*?\]\s*(.*?)(?=\.?\s*J\.\:)',
        case_text,
        re.DOTALL
    )
    if match:
        return "[{}] {}".format(match.group(1), match.group(4).strip())
    
    # Third attempt, enhanced to capture additional patterns and variations
    match = re.search(
        r'\[\s*(?:(G\.\s*R\.\s*No\.?|A\.M\.\s*No\.)\s*(L?-?\d+[-\d]*))\s*(?:.*?\.\s*)?\]\s*(THE\s*.*?|\w+\s*\w+\s*VS\.\s*\w+.*?)(?:\.\s*D\s*E\s*C\s*I\s*S\s*I\s*O\s*N\s*|R\s*E\s*S\s*O\s*L\s*U\s*T\s*I\s*O\s*N\s*)(.*?)(?=\s*J\.\:|PER\s*CURIAM\:)',
        case_text,
        re.DOTALL
    )
    if match:
        # Constructing the title to capture broader cases including different prefixes and spacing
        title = "[{}] {}".format(match.group(2), match.group(4).strip())
        return title
    
    return 'No Title Found'


# Apply the enhanced extraction function directly to each row in the 'content' column
df['case_title'] = df['content'].apply(extract_title_enhanced)


In [40]:
# Assuming 'df' has a 'case_title' column from previous extraction efforts

# Count identified and non-identified titles
identified_count = df[df['case_title'] != 'No Title Found'].shape[0]
no_title_count = df[df['case_title'] == 'No Title Found'].shape[0]

# Get samples for identified and non-identified titles
identified_samples = df[df['case_title'] != 'No Title Found']['case_title'].head(sample_limit).tolist()
no_title_samples = df[df['case_title'] == 'No Title Found']['content'].head(sample_limit).tolist()

# Displaying the results
print(f"Identified Titles: {identified_count}")
print(f"No Title Found: {no_title_count}\n")
print("Samples of Identified Titles:")
for sample in identified_samples:
    print(f"- {sample}")

print("\nSamples of Contents Without Identified Titles:")
for sample in no_title_samples:
    print(f"- {sample[:300]}")  # Show only the first 300 characters for readability

Identified Titles: 811
No Title Found: 102

Samples of Identified Titles:
- [ G.R. No. 6717. October 19, 1911 ] THE UNITED STATES, PLAINTIFF AND APPELLEE, VS. FAUSTINO MESINA, DEFENDANT AND APPELLANT. D E C I S I O N TORRES, J.:
- [ G.R. No. 6923. September 12, 1912 ] THE UNITED STATES, PLAINTIFF AND APPELLEE, VS. VALENTIN BERNABE, DEFENDANT AND APPELLANT. D E C I S I O N TORRES, J.:
- [ G.R. No. 1237. September 30, 1903 ] THE UNITED STATES, COMPLAINANT AND APPELLEE, VS. LEONARDO GUINACARAN ET AL., DEFENDANTS AND APPELLANTS. D E C I S I O N TORRES, J.:
- [ G.R. No. 6705. February 27, 1912 ] THE UNITED STATES, PLAINTIFF AND APPELLEE, VS. FELIPE SALVADOR (ALIAS APONG IPI), DEFENDANT AND APPELLANT. D E C I S I O N MORELAND, J.:
- [ G.R. No. L-12907. May 30, 1960 ] THE PEOPLE OF THE PHILIPPINES, PLAINTIFF AND APPELLEE, VS. MORO AMBAHANG, ET AL., DEFENDANTS. HANDAN AMID, INAMA MORSAN, NICOLAS CARPIO ALIAS COLAS AMBAHANG MORO, DEFENDANTS AND APPELLANTS. D E C I S I O N PER CURIAM:

Samples o